In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import torchvision
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
import time

In [2]:
flags = pd.read_csv('C:/Users/ligra/Desktop/диплом/fashion-dataset/styles.csv', error_bad_lines=False)

C:\Users\ligra\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 6044: expected 10 fields, saw 11\nSkipping line 6569: expected 10 fields, saw 11\nSkipping line 7399: expected 10 fields, saw 11\nSkipping line 7939: expected 10 fields, saw 11\nSkipping line 9026: expected 10 fields, saw 11\nSkipping line 10264: expected 10 fields, saw 11\nSkipping line 10427: expected 10 fields, saw 11\nSkipping line 10905: expected 10 fields, saw 11\nSkipping line 11373: expected 10 fields, saw 11\nSkipping line 11945: expected 10 fields, saw 11\nSkipping line 14112: expected 10 fields, saw 11\nSkipping line 14532: expected 10 fields, saw 11\nSkipping line 15076: expected 10 fields, saw 12\nSkipping line 29906: expected 10 fields, saw 11\nSkipping line 31625: expected 10 fields, saw 11

In [3]:
flags = flags.rename(columns={'masterCategory':'gen_category',
                      'subCategory':'sub_category',
                      'articleType':'type',
                      'baseColour':'color',
                      'productDisplayName':'product_name'})
flags

,id,gender,gen_category,sub_category,type,color,season,year,usage,product_name
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt
...,...,...,...,...,...,...,...,...,...,...
44419,17036,Men,Footwear,Shoes,Casual Shoes,White,Summer,2013.0,Casual,Gas Men Caddy Casual Shoe
44420,6461,Men,Footwear,Flip Flops,Flip Flops,Red,Summer,2011.0,Casual,Lotto Men's Soccer Track Flip Flop
44421,18842,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011.0,Casual,Puma Men Graphic Stellar Blue Tshirt
44422,46694,Women,Personal Care,Fragrance,Perfume and Body Mist,Blue,Spring,2017.0,Casual,Rasasi Women Blue Lady Perfume


In [4]:
img_path = 'C:/Users/ligra/Desktop/диплом/fashion-dataset/images'
images = []
for id in flags.id:
      images.append(img_path+'/'+str(id)+'.jpg') 
uniquie_article_types = flags['type'].unique()

In [5]:
flags.dropna(inplace=True, subset=['year','type'])
training_data = flags[flags['year'].astype('int') % 2 == 0]
test = flags[flags['year'].astype('int') % 2 == 1]

In [6]:
img_list = uniquie_article_types
img2num = {img:i for i, img in enumerate(img_list)}
num2img = {i:img for i, img in enumerate(img_list)}

In [7]:
transform = torchvision.transforms.Compose([
                torchvision.transforms.Resize((224, 224)),
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])

In [8]:
class img_df(Dataset):
    
    def __init__(self, dir, dataframe, transform):
        super(img_df, self).__init__()
        self.dataframe = dataframe
        self.dir = dir
        self.transform = transform
        
    def __getitem__(self, idx):
        line = self.dataframe.iloc[idx]
        cat = line.articleType
        cat_id = cat2num[cat]
        img_path = os.path.join(self.dir, str(line.id)+'.jpg')
        img = Image.open(img_path).convert('RGB')
        img_tensor = self.transform(img)
        return img_tensor, cat_id
            
    def __len__(self):
        return len(self.dataframe)